Importiamo le librerie necessarie per l'addestramento del classificatore

In [ ]:
from joblib import dump
from pandas import read_csv
from os import chdir
from sys import path
from gensim.corpora.textcorpus import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from re import sub, findall
from nltk.corpus import stopwords
from nltk import download
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix, classification_report

Eseguiamo il download del dizionario di stopwors dal modulo *nltk* e creo un set come variabile globale che contiene le parole del dizionario.

In [ ]:
download('stopwords')
stops = set(stopwords.words('english'))

Dichiariamo la funzione *text_preprocessing* e importiamo al suo interno la variabile globale stops

In [ ]:
def text_preprocessing(text):
    
    global stops

Al suo interno, svolgiamo alcune operazioni per preparare il testo prima di essere processato poi dal classificatore.
In particolare qui abbiamo eseguito alcune operazioni relative alle emoji e alla punteggiatura.

In [ ]:
    text = sub(r'(:\)|:-\)|:D|:-D)', 'happy', text)
    text = sub(r'(:\(|:-\()', 'happy', text)
    text = sub(r'o.O', 'incredulous', text)
    text = sub(r'(!!)+', 'bigExlamation', text)
    text = sub(r'!', 'Exlamation', text)
    text = sub(r'\?!\?', 'Doubtful', text)

Successivamente andiamo a sostituire tutte le parole MAIUSCOLE, con la parola stessa + "shout"parola. Questa operazione migliora ulteriormente la comprensione del codice, dal momento che, da un po' di tempo a questa parte, vi è la concezione che scrivere in maiuscolo equivale a gridare. E questa cosa la facciamo notare al classificatore.
Successivamente rendiamo tutto il testo minuscolo.

In [ ]:
    for item in findall('[A-Z]+[A-Z]+', text):
        text = sub(item, f'{item.lower()} shout{item.lower()}', text, count=1)
    text = text.lower()

Creiamo un generatore *filtered_word* che restituirà separatamente tutte le parole del testo che non sono stopwords.
Dopodichè le riuniamo nuovamente nella variabile text.

In [ ]:
    filtered_words = (word for word in text.split() if word not in stops)
    text = ' '.join(filtered_words)

Utilizzando alcune funzioni presenti in *gensim* andiamo ad effettuare alcune operazioni come la rimozione della punteggiatura, la rimozione dei numeri, la rimozione delle parole più corte di 3 caratteri, la rimozione degli spazi bianchi multipli e infine la funzione restituisce il testo "stemmatizzato"

In [ ]:
    text = strip_punctuation(text)
    text = strip_numeric(text)
    text = strip_short(text, minsize=3)
    text = strip_multiple_whitespaces(text)
    return stem_text(text)

Dichiariamo la funzione *main*

In [ ]:
def main():

Al suo interno importiamo la variabile globale *stops* e l'istruzione per poter spostarci nella cartella di lavoro in cui è presente il file che viene eseguito.

In [ ]:
    global stops
    chdir(path[0])

Inizializziamo un dataframe leggendolo dal file .csv di addestramento, che contiene le colonne *'Review'* e *'Sentiment'*. A tutta la colonna relativa alle recensioni andiamo ad applicare la funzione di preprocessing.

In [ ]:
    df = read_csv('IMDB Dataset.csv')
    df['Review'] = df['Review'].map(text_preprocessing)

Dividiamo il dataset in variabili indipendenti (x, le recensioni) e dipendenti (y, il risultato associato alla recensione, ovvvero **negativo** o **positivo**). Il parametro *test_size* sta ad indicare che la dimensione del dataset di test sarà 1/3 del totale, e di conseguenza la partizione relativa al dataset di addestramento 2/3 del totale.

In [ ]:
    X_train, X_test, y_train, y_test = train_test_split(df['Review'], df['Sentiment'], test_size=0.33, random_state=10)

Creiamo un oggetto di tipo **CountVectorizer**

In [ ]:
    count_vect = CountVectorizer()

Su cui viene invocato il metodo *fit_transform* per la creazione della **matrice document-term**

In [ ]:
    X_train_matrix = count_vect.fit_transform(X_train)

Creiamo quindi un oggetto di tipo **TfidfTransformer** su cui verrà invocato il metodo *fit_transform* che restituirà un array con i vari valori TF-IDF.

In [ ]:
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_matrix)

Creiamo il classificatore e lo addestriamo con il dataset di test delle recensioni e dei sentimenti (**X** e **y**)

In [ ]:
    clf = MultinomialNB()
    clf.fit(X_train_tfidf, y_train)

Addestrato il classificatore sui 2/3 del Dataset, proviamo a eseguire il test con la restante parte del totale. Procediamo quindi alla stessa maniera di prima, utilizzando però le variabili relative al test e non all'addestramento e il metodo *transform* invece di *fit_transform*.
È importante utilizzare lo stesso oggetto **CountVectorizer** e **TfidfTransformer** utilizzati per l'addestramento.

In [ ]:
    X_test_matrix = count_vect.transform(X_test)
    X_test_tfidf = tfidf_transformer.transform(X_test_matrix)


Infine creo un array con i risultati del test.

In [ ]:
    predicted = clf.predict(X_test_tfidf)

Possiamo poi eventualmente stampare la matrice di confusione e le statistiche relative al classificatore per valutarne l'efficienza.

In [ ]:
    print(confusion_matrix(y_test,predicted))
    print(classification_report(y_test, predicted))

Inoltre, per poter utilizzare il classificatore in un altro script di Python o a distanza di tempo, per non eseguire di nuovo tutto l'addestramento partendo dal Dataset iniziale, esportiamo i tre oggetti necessari (**Classificatore**, **TfidfTransformer** e **CountVectorizer**) in una cartella del progetto creata appositamente. Questo è possibile grazie al modulo *joblib*. Si può fare altettanto con il modulo *pickle*.

In [ ]:
    dump(clf, './Objects/clf.pkl') 
    dump(count_vect, './Objects/count_vect.pkl')
    dump(tfidf_transformer, './Objects/tfidf_transformer.pkl')

In [ ]:
if __name__ == '__main__':
    main()